In [1]:
!pip install pint

import pandas as pd
import re
from pint import UnitRegistry
ureg = UnitRegistry()
from tqdm import notebook

     |████████████████████████████████| 194kB 2.7MB/s 


In [2]:
I_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/INGR_FINAL_COMPL_MERGE.csv', index_col=0).rename(columns={'modifier': 'modifier1'})
I_DF

,modifier1,name,quantity,sub_qty,sub_unit,unit,recipe_index,count,fdc_id
0,divided,canola oil,2.00,NaN,NaN,teaspoons,0,987.0,172336.0
1,NaN,diced apples,2.00,NaN,NaN,cups,0,15.0,171689.0
2,divided,white sugar,2.00,NaN,NaN,tablespoons,0,18279.0,343930.0
3,NaN,ground cinnamon,2.00,NaN,NaN,teaspoons,0,6561.0,171320.0
4,NaN,milk,0.75,NaN,NaN,cup,0,8144.0,171265.0
...,...,...,...,...,...,...,...,...,...
651088,warmed,milk,0.75,NaN,NaN,cup,68633,8144.0,171265.0
651089,separated,eggs,2.00,NaN,NaN,large,68633,11421.0,339003.0
651090,melted,unsalted butter,2.00,NaN,NaN,tablespoons,68633,2030.0,343816.0
651091,or as needed,vegetable oil,1.00,NaN,NaN,teaspoon,68633,6748.0,343866.0


In [0]:
F_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/FoodData_Central/food.csv')

In [0]:
P_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/FoodData_Central/food_portion.csv')

In [0]:
I_F_P_DF = I_DF.merge(F_DF, on='fdc_id', how='left').merge(P_DF, on='fdc_id', how='left')

In [0]:
SL_DF = I_F_P_DF[I_F_P_DF.data_type == 'sr_legacy_food']
SL_DF = SL_DF.drop(columns=['food_category_id', 'publication_date', 'id',
                    'seq_num', 'measure_unit_id', 'portion_description', 'data_points', 'footnote', 'min_year_acquired'])

In [0]:
from fractions import Fraction

def fraction_to_decimal(string):
  return float(sum(Fraction(s) for s in string.split()))

def to_decimal(string):
  try:
    if re.match('.*-.*', string):
      left = fraction_to_decimal(string.split('-')[0])
      right = fraction_to_decimal(string.split('-')[1])
      return left + right
    return fraction_to_decimal(string)
  except:
    return 0

In [0]:
# SL_DF['quantity'] = SL_DF['quantity'].replace(to_replace=r'n', value='', regex=True)
# SL_DF['quantity'] = SL_DF['quantity'].replace(to_replace=r'a', value='', regex=True)
SL_DF['quantity'] = SL_DF['quantity'].astype(str).apply(to_decimal).astype(float)
SL_DF['amount'] = SL_DF['amount'].astype(str).apply(to_decimal).astype(float)
SL_DF['sub_qty'] = SL_DF['sub_qty'].astype(str).apply(to_decimal).astype(float)

In [0]:
SL_DF = SL_DF.astype({'unit':str, 'quantity': float, 'sub_unit': str, 'sub_qty': float, 'modifier':str, 'amount':float})
SL_DF['household_modifier'] = ''

In [10]:
def parse_portion_modifier(df):
  for index, row in notebook.tqdm(df.iterrows(), total=len(df)):
    if row['modifier'] != '':
      result = re.findall(',.*', row['modifier'])
      if result != None:
        if len(result) != 0:
          result = result[0]
          df.at[index, 'modifier'] = row['modifier'].replace(result, '')
          df.at[index, 'household_modifier'] = result.lstrip(', ')
          # print(result)
  return df

SL_DF = parse_portion_modifier(SL_DF)

In [0]:
I_DF = SL_DF[['name', 'quantity', 'unit', 'recipe_index','fdc_id', 'sub_qty','sub_unit']].drop_duplicates()
I_DF['grams'] = ''

In [12]:

def is_mass(unit):
  try:
    measure = 1 * ureg(unit)
    if (measure.check('[mass]')): return True
    else: return False
  except:
    return False

def is_volume(unit):
  try:
    measure = 1 * ureg(unit)
    if (measure.check('[volume]')): return True
    else: return False
  except:
    return False

def mass_to_grams(unit, qty):
  measure = qty * ureg(unit)
  return measure.to('grams').magnitude

def volume_to_grams(i_unit, i_qty, h_unit, h_qty, serving_size):
  h_measure = h_qty * ureg(h_unit)
  i_measure = i_qty * ureg(i_unit)
  s_measure = serving_size * ureg('grams')
  i_to_h = i_measure.to(h_measure)
  to_grams = i_to_h.magnitude / h_measure.magnitude
  grams = to_grams * serving_size
  return grams

def ballpark_volume_to_grams(unit, qty):
  measure = qty * ureg(unit)
  return measure.to('ml').magnitude


# def volume_to_grams(i_unit, i_qty, h_unit, h_qty):

def get_value(row, asset):
  try:
    return row[asset].to_list()[0] 
  except:
    return ''

def val_isna(value):
  try:
    if(math.isnan(value)): return True
    else: return False
  except:
    re

def not_specified(df):
  for index, row in df.iterrows():
    if row.portion_description == 'Quantity not specified':
      return row['gram_weight']

def get_matching_portion_row(unit, h_rows):
  for index, row in h_rows.iterrows():
    if is_volume(unit) and is_volume(row['modifier']):
      return row
    # elif is_mass(unit) and is_mass(row['household_unit']):
    #   return row
    elif unit in row['modifier']:
      return row

  return pd.DataFrame({'A' : []})


UNCOMMON_UNITS = {
    'cup': ['jar','jars','can', 'cans'],'bulb':['bulb', 'bulbs'],
    'head': ['head', 'heads'], 'pie shell':['shell', 'pie shell','shells','pie shells'],
    'gram': ['drop', 'drops', 'fluid']
}

def unit_in_ingr_name(string):
  for key in UNCOMMON_UNITS:
    for value in UNCOMMON_UNITS[key]:
      if value in string:
        return key
      
  return 'nan'


# get_matching_portion_row('sdfds', SF_DF[(SF_DF.fdc_id == 342104.0) & (SF_DF.recipe_index == 295)]).empty
# unit_in_ingr_name('bulb fennel')
# SF_DF[(SF_DF.fdc_id == 343873) & (SF_DF.recipe_index == 0)]
# volume_to_grams('quart', 2, 'cup', 1, 200)
ballpark_volume_to_grams('cup', 1)

236.58823649999994

In [0]:
def no_unit_nor_qty(ingr, h_rows):
  # If only one portion
  if len(h_rows) == 1:
    # use the weight for that portion
    return h_rows.head(1)['gram_weight'].to_list()[0]
  # try dash (salt and pepper = dash on portion)
  matching_row = get_matching_portion_row('dash', h_rows)
  if not matching_row.empty:
    return matching_row['gram_weight']
  # else if small, medium, or large are in portions
  matching_small = get_matching_portion_row('small', h_rows)
  matching_medium = get_matching_portion_row('medium', h_rows)
  matching_large = get_matching_portion_row('large', h_rows)
  matching_fruit = get_matching_portion_row('fruit', h_rows)
  if not matching_small.empty: return matching_small['gram_weight']
  elif not matching_medium.empty: return matching_medium['gram_weight']
  elif not matching_large.empty: return matching_large['gram_weight']
  elif not matching_fruit.empty: return matching_fruit['gram_weight']
  else:
    # use first portion on list
    return h_rows.head(1)['gram_weight'].to_list()[0]


In [0]:
UNCOMMON_UNITS = {
    'dash': ['pinch', 'pinches'], 'pinches': ['pinch', 'pinches'], 'dash': ['dash','dashes'], 'dashes':['dash','dashes'], 
    'slice':['slice', 'slices'], 'slices':['slice', 'slices'], 'bunch':['bunch', 'bunches'], 'bunches':['bunch', 'bunches'],
    'cube':['cube', 'cubes'], 'cubes':['cube', 'cubes'], 'clove':['clove', 'cloves'], 'cloves':['clove', 'cloves'], 
    'leaf':['leaf', 'leaves'], 'leaves':['leaf', 'leaves'], 'sprig':['sprig','sprigs'],'sprigs':['sprig','sprigs'],
    'chop':['chop', 'chops'], 'chops':['chop', 'chops'], 'crust':['crust', 'crusts'], 'crusts':['crust', 'crusts'],
    'stalk':['stalk','stalks'],'stalks':['stalk','stalks'], 'head':['head', 'heads'], 'heads':['head', 'heads'], 
    'small':['small'], 'medium':['medium'],'large':['large'],'tortilla':['tortilla','tortillas'],'tortillas':['tortilla','tortillas'],
    'yield': ['yield', 'yields'], 'yields': ['yield', 'yields']
}

def unit_in(string):
  keys = []
  string = string.split()
  for key in UNCOMMON_UNITS:
    for value in UNCOMMON_UNITS[key]:
      if value in string:
        keys.append(key)
  return keys

def no_unit(ingr, h_rows):
  # display(h_rows)
  # if one of the uncommon units in portions (try singular or plural for ingredient and portion):
  for index, row in h_rows.iterrows():
    units = unit_in(row['modifier'])
    if len(units) != 0:
      for unit in units:
        # print('matching unit: ',unit)
        matching_row = get_matching_portion_row(unit, h_rows)
        if not matching_row.empty:
          return matching_row['gram_weight'] * ingr['quantity']
  # if nothing returns, return first portion * qty
  return h_rows.head(1)['gram_weight'].to_list()[0] * ingr['quantity']



In [0]:

def not_specified(ingr, h_rows):
  # If no unit nor qty:
  if ingr['unit'] == 'nan' and ingr['quantity'] == 0:
    return no_unit_nor_qty(ingr, h_rows)
  # if no unit:
  if ingr['unit'] == 'nan' and ingr['quantity'] != 0:
    return no_unit(ingr, h_rows)
  # grams = special case(inr_h_rows)
  return h_rows.head(1)['gram_weight'].to_list()[0] * ingr['quantity']


In [16]:
def extract_grams(i_df, p_df):
  for index, row in notebook.tqdm(i_df.iterrows(), total=len(i_df)):
    h_rows = p_df[(p_df.fdc_id == row['fdc_id']) & (p_df.recipe_index == row['recipe_index'])]
    grams = 0
    try:
      # CASE 1: If the ingredient is of mass type: WORKS
      if is_mass(row['unit']):
        # Convert directly to grams
        grams = mass_to_grams(row['unit'], row['quantity'])
      
      # CASE 2: If there is a subunit WORKS
      elif row['sub_unit'] != 'nan':
          # If it is mass type
          if is_mass(row['sub_unit']):
            # Multiply subunit by qty and convert directly to grams
            grams = mass_to_grams(row['sub_unit'], row['quantity'] * row['sub_qty']) 
          # Else if subunit is 'fluid':
          elif 'fluid' in row['sub_unit']:
            # use oz
            grams = mass_to_grams('ounce', row['quantity'] * row['sub_qty'])
          else:
            # Use not specified
            # grams = not_specified(h_rows)
            grams = 9999

      # CASE 4: If there is a qty but no unit for ingredient ('1', '') ('1', 'lemon') AND no subunit WORKS
      elif row['quantity'] != 0 and row['unit'] == 'nan' and row['sub_qty'] == 0 and row['sub_unit'] == 'nan':
        # CASE 2.1: Check if unit is in the ingr name (from our dict of units) and convert accordingly
          # Convert the unit in name to common unit
          unit = unit_in_ingr_name(row['name'])
          # If it is in the ingredient name:
          if unit != 'nan':
            # Find the matching unit type in portion rows
            matching_row = get_matching_portion_row(unit, h_rows)

            # If there was a matching row found:
            if not matching_row.empty:
              # Multiply qty by the gram weight of matching row
              grams = row['quantity'] * matching_row['gram_weight']
          # Else
          else:
            # Multiply ingredient quantity with not specified grams
            grams = not_specified(row, h_rows)
            # grams = 9999
      # CASE 5: Ingredient unit is of volume type: WORKS
      elif is_volume(row['unit']):
        # Find portion row with matching volume type
        # display(h_rows)
        matching_row = get_matching_portion_row(row['unit'], h_rows)
        # If there was a matching row found:
        if not matching_row.empty:
          grams = volume_to_grams(row['unit'], row['quantity'], 
                                  matching_row['modifier'], matching_row['amount'], matching_row['gram_weight'])
        else: 
          # Convert using ballpark volume to grams conversion
          grams = ballpark_volume_to_grams(row['unit'], row['quantity'])
      # If grams still is 0 at this point, just use not specified
      if grams == 0:
        grams = not_specified(row, h_rows)
        # grams = 9999

      
    except KeyboardInterrupt:
      raise
    except Exception as e:
      grams = not_specified(row, h_rows)
      # raise
      # grams = 9999
      print(e)
    i_df.at[index, 'grams'] = grams
    
  
  return i_df


ingr_df = extract_grams(I_DF, SL_DF)

# I_DF[(I_DF.quantity != 0) & (I_DF.unit.isna()) & (I_DF.sub_qty == 0) & I_DF.sub_unit.isna() & (I_DF.name != 'eggs') & (I_DF.name != 'egg')]
# row['quantity'] != 0 and row['unit'] == 'nan' and row['sub_qty'] == 0 and row['sub_unit'] == 'nan'
# I_DF[(I_DF.quantity != 0.0) & (I_DF.unit == 'nan') & (I_DF.sub_qty == 0) & (I_DF.sub_unit == 'nan') & (I_DF.name != 'eggs') & (I_DF.name != 'egg')]
# I_DF[I_DF.unit.apply(is_volume)]



Streaming output truncated to the last 5000 lines.
'chopped' is not defined in the unit registry
'cherry' is not defined in the unit registry
'cherry' is not defined in the unit registry
'packed' is not defined in the unit registry
Cannot convert from 'cup' ([length] ** 3) to 'cup * femtoliter * ounce' ([length] ** 6 * [mass])
'packed' is not defined in the unit registry
'chopped' is not defined in the unit registry
'packed' is not defined in the unit registry
'chopped' is not defined in the unit registry
'packed' is not defined in the unit registry
'chopped' is not defined in the unit registry
'chopped' is not defined in the unit registry
'chopped' is not defined in the unit registry
'packed' is not defined in the unit registry
'chopped' is not defined in the unit registry
'chopped' is not defined in the unit registry
'chopped' is not defined in the unit registry
'chopped' is not defined in the unit registry
'chopped' is not defined in the unit registry
'chopped' is not defined in the

In [0]:
ingr_df.to_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/SR_LEGACY_INGR_MASS.csv')